In [1]:
! pip install transformers
! pip install torch

In [114]:
from transformers import BertTokenizer, BertModel
import torch

# BERT_MODEL='bert-base-cased'
BERT_MODEL='dmis-lab/biobert-base-cased-v1.1'

tokenizer = BertTokenizer.from_pretrained(BERT_MODEL, do_lower_case=False)
model = BertModel.from_pretrained(BERT_MODEL, output_hidden_states = True)

In [3]:
def get_hidden_states(model,tokenizer,marked_text):
    tokenized_text = tokenizer.tokenize(marked_text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    
    segments_ids = [1] * len(tokenized_text)

    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
    
    with torch.no_grad():
        outputs = model(tokens_tensor, segments_tensors)
        hidden_states = outputs[2]
        return hidden_states

In [4]:
def get_embeddings(text):
    marked_text = "[CLS] " + text + " [SEP]"
    
    hidden_states = get_hidden_states(model, tokenizer, marked_text)
    token_embeddings = torch.stack(hidden_states, dim=0)
    token_embeddings = torch.squeeze(token_embeddings, dim=1)
    token_embeddings = token_embeddings.permute(1,0,2)
    
    return token_embeddings

In [115]:
input_sentence = "A 58-year-old African-American woman presents to the ER with episodic pressing/burning anterior chest pain that began two days earlier for the first time in her life. The pain started while she was walking, radiates to the back, and is accompanied by nausea, diaphoresis and mild dyspnea, but is not increased on inspiration. The latest episode of pain ended half an hour prior to her arrival. She is known to have hypertension and obesity. She denies smoking, diabetes, hypercholesterolemia, or a family history of heart disease. She currently takes no medications. Physical examination is normal. The EKG shows nonspecific changes."
tokenized_input = tokenizer.tokenize(input_sentence)

In [116]:
import pandas as pd

# print(type(tokenized_input))
# tokenized_input.remove("##")
# print(tokenized_input)

tokenized_input_pd = pd.Index(tokenized_input)


token_counts = tokenized_input_pd.value_counts()
token_indexes = token_counts.keys()
token_counts

.        8
,        7
is       4
to       4
She      3
        ..
a        1
heart    1
##is     1
first    1
ER       1
Length: 113, dtype: int64

In [118]:
first_embeddings = []
last_embeddings = []

with open("embeddings/label.tsv", "w") as labels_file:
    for token in tokenized_input:
        if (not token.startswith("##")):
            marked_text = "[CLS] " + token + " [SEP]"
#             print(type(token_counts[token]))
            print(token + '\t' + str(2), file=labels_file)

            token_embeddings = get_embeddings(marked_text)
        
            first_embedding_layer = token_embeddings[1][1]
            last_embedding_layer = token_embeddings[1][12]

            tsv_rows = ''
            for e in last_embedding_layer:
                tsv_rows += str(e.item()) + '\t'
            last_embeddings.append(tsv_rows)
            
            tsv_rows = ''
            for e in first_embedding_layer:
                tsv_rows += str(e.item()) + '\t'
            first_embeddings.append(tsv_rows)

In [92]:
import csv

first_embeddings = []
last_embeddings = []
valid_tokens_indexes = []
valid_tokens_counts = []
# data = { token_indexes, token_counts }
# print(data)
# a = pd.Index(token_indexes, token_counts)
# a.to_csv("embeddings/label.tsv", header=["word", "count"], index=False)



# print(len(tokenized_input))

i = 0

for token in tokenized_input:
    if (not token.startswith("##")):
        i= i + 1
        marked_text = "[CLS] " + token + " [SEP]"

#             writer = csv.writer(labels_file)
#             writer.writerow([token, token_counts[token]])

#         token_indexes.append()
        
        valid_tokens_indexes.append(token)
        valid_tokens_counts.append(token_counts[token])

        token_embeddings = get_embeddings(marked_text)
        
        first_embedding_layer = token_embeddings[1][1]
        last_embedding_layer = token_embeddings[1][12]

        tsv_rows = ''
        for e in last_embedding_layer:
            tsv_rows += str(e.item()) + '\t'
        last_embeddings.append(tsv_rows)
            
        tsv_rows = ''
        for e in first_embedding_layer:
            tsv_rows += str(e.item()) + '\t'
        first_embeddings.append(tsv_rows)
i

122

In [119]:
with open("embeddings/biobert/first_embedding.tsv", "w") as f:
    for e in first_embeddings:
        print (e, file=f)
        
with open("embeddings/biobert/last_embedding.tsv", "w") as f:
    for e in last_embeddings:
        print (e, file=f)
        
# data = {'wotttrd':  valid_tokens_indexes,
#         'count': valid_tokens_counts
#        }
# data_tuples = list(zip(valid_tokens_indexes,valid_tokens_counts))
# df = pd.DataFrame(data_tuples)
# df.to_csv("embeddings/label.tsv", index=False, sep="\t")


In [98]:
len(first_embeddings)


122

In [99]:
data_tuples

[('A', 1),
 ('58', 1),
 ('-', 3),
 ('year', 1),
 ('-', 3),
 ('old', 1),
 ('African', 1),
 ('-', 3),
 ('American', 1),
 ('woman', 1),
 ('presents', 1),
 ('to', 4),
 ('the', 3),
 ('ER', 1),
 ('with', 1),
 ('e', 1),
 ('pressing', 1),
 ('/', 1),
 ('burning', 1),
 ('anterior', 1),
 ('chest', 1),
 ('pain', 3),
 ('that', 1),
 ('began', 1),
 ('two', 1),
 ('days', 1),
 ('earlier', 1),
 ('for', 1),
 ('the', 3),
 ('first', 1),
 ('time', 1),
 ('in', 1),
 ('her', 2),
 ('life', 1),
 ('.', 8),
 ('The', 3),
 ('pain', 3),
 ('started', 1),
 ('while', 1),
 ('she', 1),
 ('was', 1),
 ('walking', 1),
 (',', 7),
 ('r', 1),
 ('to', 4),
 ('the', 3),
 ('back', 1),
 (',', 7),
 ('and', 3),
 ('is', 4),
 ('accompanied', 1),
 ('by', 1),
 ('nausea', 1),
 (',', 7),
 ('di', 1),
 ('and', 3),
 ('mild', 1),
 ('d', 1),
 (',', 7),
 ('but', 1),
 ('is', 4),
 ('not', 1),
 ('increased', 1),
 ('on', 1),
 ('inspiration', 1),
 ('.', 8),
 ('The', 3),
 ('latest', 1),
 ('episode', 1),
 ('of', 2),
 ('pain', 3),
 ('ended', 1),
 ('half'